###  2024-2025 복지 급여 PDF 파싱 및 JSON 생성
- 연도별 복지급여 비교 기능
- 사용자 소득 기준 급여 자격 확인

In [3]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import fitz
from typing import List

load_dotenv()

True

In [4]:
def extract_pdf_data(pdf_path: str, pages: List[int]):
    doc = fitz.open(pdf_path)
    text = ""
    tables = []
    
    for page_num in pages:
        if page_num <= len(doc):
            page = doc[page_num - 1]
            text += f"\n=== 페이지 {page_num} ===\n{page.get_text()}\n"
            
            try:
                page_tables = page.find_tables()
                for table in page_tables:
                    data = table.extract()
                    if data:
                        tables.append(data)
            except:
                pass
    
    doc.close()
    return text, tables

In [5]:
def create_prompt(year: int, text: str, tables: List) -> str:
    return f"""{year}년 복지 PDF 파서입니다.

다음 JSON 구조로 데이터를 추출하세요:

반환 형식:
- year: {year}
- income_standards 객체:
  - 중위소득 객체:
    - 단위: "원/월"
    - 가구별 금액 객체: 1인~7인 가구별 금액 (숫자)
  - 맞춤형 기초생활 급여 객체:
    - 생계급여, 의료급여, 주거급여, 교육급여 각각:
      - 기준: 중위소득 비율
      - 단위: "원/월"
      - 가구별 금액 객체: 1인~7인 가구별 금액
- eligibility_categories 객체:
  - 부양의무자, 차상위계층 등 정의

주의사항:
- 금액은 원 단위 정수만
- 가구 키는 "1인 가구", "2인 가구" 형식
- 7인가구 데이터가 표에 없으면 텍스트에서 찾기
- 유효한 JSON만 반환

PDF 텍스트:
{text}

표 데이터:
{str(tables)}

JSON 응답:"""

In [6]:
# 전역 변수로 데이터 저장

text_data = None
tables_data = None
prompt_text = None
response_data = None
result_data = None
year = 2024
pdf_path = "../data/pdfs/2024+나에게+힘이+되는+복지서비스+(통합권).pdf"

In [7]:
#  PDF 데이터 추출
text_data, tables_data = extract_pdf_data(pdf_path, [12, 13])


In [8]:
# 프롬프트 생성
prompt_text = create_prompt(year, text_data, tables_data)


In [9]:
# LLM 실행
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
response_data = llm.invoke(prompt_text)

In [10]:
# JSON 파싱 및 저장
result_text = response_data.content
start_idx = result_text.find('{')
end_idx = result_text.rfind('}') + 1
json_str = result_text[start_idx:end_idx]
result_data = json.loads(json_str)

os.makedirs("../data/outputs", exist_ok=True)
with open(f"../data/outputs/welfare_{year}.json", "w", encoding="utf-8") as f:
    json.dump(result_data, f, ensure_ascii=False, indent=2)

print(f"welfare_{year}.json 저장 완료")

welfare_2024.json 저장 완료


In [12]:
# 2025년 데이터 처리
year = 2025
pdf_path = "../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"

In [13]:
result_text = response_data.content
start_idx = result_text.find('{')
end_idx = result_text.rfind('}') + 1
json_str = result_text[start_idx:end_idx]
result_data = json.loads(json_str)

os.makedirs("../data/outputs", exist_ok=True)
with open(f"../data/outputs/welfare_{year}.json", "w", encoding="utf-8") as f:
    json.dump(result_data, f, ensure_ascii=False, indent=2)

print(f"welfare_{year}.json 저장 완료")

welfare_2025.json 저장 완료
